# Pandas Express
# TTC-Delays project
Team Members: 

Gaspare Castagna

Amirhossein Heidari

Avery Hoffer

Michael Metallo

In [1]:
import numpy as numpy
import pandas as pd
import os
import numpy as np
!pip install python-Levenshtein
!pip install rapidfuzz
import Levenshtein
import re

Import, read, concat bus delay data

In [2]:
def read_excel_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    data_frames = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [28]:
df_2014 = read_excel_sheets('bus_2014.xlsx')
df_2015 = read_excel_sheets('bus_2015.xlsx') 
df_2016 = read_excel_sheets('bus_2016.xlsx') 
df_2017 = read_excel_sheets('bus_2017.xlsx') 
df_2018 = read_excel_sheets('bus_2018.xlsx')
df_2018 = df_2018.iloc[:, :-1]
df_2019 = read_excel_sheets('bus_2019.xlsx')
df_2019 = df_2019.iloc[:, :-3]
df_2020 = read_excel_sheets('bus_2020.xlsx')
df_2020 = df_2020.iloc[:, :-3]
df_2020.rename(columns={'Gap': 'Min Gap', 'Delay': 'Min Delay'}, inplace=True)
df_2021 = read_excel_sheets('bus_2021.xlsx')
df_2021.rename(columns={'Date': 'Report Date'}, inplace=True)
df_2021 = df_2021.iloc[:, :-3]
df_2022 = read_excel_sheets('bus_2022.xlsx')
df_2022.rename(columns={'Date': 'Report Date'}, inplace=True)

bus_delay_list = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022]
df_bus_delay = pd.concat(bus_delay_list)

df_bus_delay['YTime'] = pd.to_datetime(df_bus_delay['Time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M')
df_bus_delay['DateTime'] = df_bus_delay['Report Date'].dt.strftime('%Y-%m-%d')+'-'+ df_bus_delay['YTime']
df_bus_delay['DateTime'] = pd.to_datetime(df_bus_delay['DateTime'])

drop_columns = ['YTime','Time','Report Date','Direction']
df_bus_delay = df_bus_delay.drop(drop_columns, axis = 1)
df_bus_delay = df_bus_delay.dropna(subset=['Route'])

df_bus_delay['copy'] = df_bus_delay['Location'].str.lower()
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('vp','victoria park')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace(' to ','$').str.replace('b/w','').str.replace('w/b','').str.replace('e/b','').str.replace('s/b','').str.replace('n/b','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('stn','station').str.replace(' and ','&').str.replace('/','&').str.replace(' ','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('(','').str.replace(')','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('@','&')
df_bus_delay['copy'] = df_bus_delay['copy'].astype(str)
df_bus_delay = df_bus_delay[~df_bus_delay['copy'].str.isdigit()]
df_bus_delay['copy'] = df_bus_delay['copy'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)





In the section below I am testing the fuzz score function we will be using:

In [7]:
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('@','&')


In [29]:
stops = pd.read_csv('stops.txt')
stops['cleaned_stops'] = stops['stop_name'].str.replace('at', '&').str.replace('Rd', '').str.replace('Ave', '').str.replace('Dr', '').str.replace('St', '')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace('Side', '').str.replace('@','&')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace(' ', '').str.lower()
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace('(north)', '').str.replace('(south)', '').str.replace('(east)', '').str.replace('(west)', '')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace('(mall)', '')
stops['cleaned_stops'] = stops['cleaned_stops'].apply(lambda x: re.sub(r'\(.*?\)', '', x))
stops['cleaned_stops'] = stops['cleaned_stops'].astype(str).str.replace('\d+', '', regex=True)
stops['cleaned_stops'] = stops['cleaned_stops'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)

In [27]:
east_mall = stops[stops['stop_name'].str.lower().str.contains('east mall')]
west_mall = stops[stops['stop_name'].str.lower().str.contains('west mall')]
hwy = stops[stops['stop_name'].str.lower().str.contains('hwy')]
print(east_mall['stop_name'].nunique())
print(west_mall['stop_name'].nunique())
len(west_mall)
len(east_mall)

46
41


56

In [58]:
unique_delays = df_bus_delay['copy'].unique()
entire_route = df_bus_delay['copy'].str.contains('route').sum()
contains_route = df_bus_delay[df_bus_delay['copy'].str.contains('route')]['copy'].unique()
filtered_df = df_bus_delay[~df_bus_delay['copy'].str.contains('route')].copy()
weird_symbols = stops['cleaned_stops'].str.contains('[\)]').sum()

contains_wird_symbol = stops[stops['cleaned_stops'].str.contains('[\)]')]['cleaned_stops'].unique()

In [30]:
# remove items not checked from df_bus delays
df_bus_delay_fuzz = df_bus_delay[~df_bus_delay['copy'].str.contains('route')].copy()
df_bus_delay_fuzz = df_bus_delay_fuzz[~df_bus_delay_fuzz['copy'].isin(stops['cleaned_stops'])]
df_bus_delay_fuzz = df_bus_delay_fuzz[df_bus_delay_fuzz['copy'].str.contains('&')]

In [31]:
unique_values = pd.unique(df_bus_delay_fuzz['copy'])

# Create a new DataFrame with a column of unique values
df_delay_fuzz = pd.DataFrame({
    'fuzz_locations': unique_values
})

In [13]:
len(df_delay_fuzz)

37547

This function takes in a string "location" and compares it to each entry in a list of strings "entry". 
It returns a string corresponding to the "best" match for the input string from the list of strings.
We can adjust the threshold to avoid false positives. Strings with no matches above the threshold will not be modified.

In [220]:
def compute_similarity(string1, string2):
    distance = Levenshtein.distance(string1, string2)
    max_length = max(len(string1), len(string2))
    if max_length == 0:
        return 100
    else:
        return 100 - (distance / max_length) * 100
    
def synchronize_locations_ls(location, reference_list):
    # 1) create an array with the similarity scores between the location and each entry in reference list
    similarity_scores = []
    for item in reference_list:
        string=str(item)
        print(location)
        print(string)
        print(compute_similarity(location[0], string))
        similarity_scores.append(compute_similarity(location, string))
    # 2) find the highest score in this list
    max_score = max(similarity_scores)
    # 3) if the score is above the threshold, return the string from the list that corresponds to the highest score
    interpolated = reference_list[similarity_scores.index(max_score)] if max_score <= 70 else "user check"
    return interpolated

In [6]:
from fuzzywuzzy import process

def find_closest_match(row, reference_list):
    match = process.extractOne(row, reference_list)
    print('completed one row')
    return match[0]

In [32]:
from rapidfuzz import process
def find_closest_match_rapid(row, reference_list, threshold=50):
    
    match = process.extractOne(row, reference_list)

    if match[1] < threshold:
        return "check"
    
    return match[0]

In [36]:
stops[stops['cleaned_stops']=='brimley']

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,cleaned_stops
1732,2481,4814,2278 Brimley Rd,NaN,43.797048,-79.272874,NaN,NaN,NaN,NaN,NaN,1,brimley
9161,24207,15818,1965 Brimley Rd,NaN,43.785127,-79.266951,NaN,NaN,NaN,NaN,NaN,1,brimley
9162,24208,15819,1994 Brimley Rd,NaN,43.785605,-79.267352,NaN,NaN,NaN,NaN,NaN,1,brimley


In [24]:
location = 'yonge&steeleswest'
reference_list = ['yonge&steeles', 'yorkmillsstation','victoriapark&danforth']

In [25]:
print(find_closest_match_rapid(location, reference_list))

Completed one row
yonge&steeles


In [33]:
df_delay_fuzz['fuzzed'] = df_delay_fuzz['fuzz_locations'].apply(find_closest_match_rapid, args=(stops['cleaned_stops'],))

In [34]:
df_delay_fuzz.head(20)

,fuzz_locations,fuzzed
0,lawrence&warden,warden
1,lawrence&scarboroughgolfclubrd,lawrenceeast&scarboroughgolfclub
2,danforthave&victoriapark,danforth&victoriapark
3,victoria&park&danforthave.,danforth
4,charlottetown&lawrence,ion&lawrence
5,brimley&lawrence,brimley
6,bloor&lansdowne,lansdowne
7,mccowan&steeles,mccowan
8,bloor&oldmill,bloorwest&oldmill
9,danforth&danforth,danforth


from fuzzywuzzy import fuzz

def synchronize_locations_fuzz(location, reference_list):
    # 1) Create an array with the similarity scores between the location and each entry in the reference list
    similarity_scores = [fuzz.ratio(location, string) for string in reference_list]
    # 2) Find the highest score in this list
    max_score = max(similarity_scores)
    # 3) If the score is above the threshold, return the string from the list that corresponds to the highest score
    interpolated = reference_list[similarity_scores.index(max_score)] if max_score >= 40 else "user check"
    return interpolated

In [210]:
locations = ['yorkmillsstation']
reference = ['yonge&steeles', 'yorkmillsstation', 'davenport&dufferin']

In [211]:
synchronized_values = synchronize_locations_ls(locations, reference)

['yorkmillsstation']
yonge&steeles
25.0
['yorkmillsstation']
yorkmillsstation
100.0
['yorkmillsstation']
davenport&dufferin
5.555555555555557


In [208]:
synchronized_values

'yonge&steeles'

In [221]:
np_copy = list(df_bus_delay['copy'])
np_stops = list(stops['cleaned_stops'])

In [ ]:
df_bus_delay.head(50)

In [82]:
def synchronize_locations_np(location, reference_list):
    synchronized_list = []
    for loc in location:
        fuzz_list = np.array([fuzz.ratio(loc, string) for string in reference_list])
        max_index = np.argmax(fuzz_list)
        if fuzz_list[max_index] >= 65:
            interpolated = reference_list[max_index]
        else:
            interpolated = "user check"
        synchronized_list.append(interpolated)
    return synchronized_list

In [83]:
synchronized_values = synchronize_locations_np(df_bus_delay['copy'].to_numpy(), stops['cleaned_stops'].to_numpy())
df_bus_delay['synchronized'] = synchronized_values

KeyboardInterrupt: 

In [75]:
def synchronize_locations(location, reference_list):
    fuzz_list = [fuzz.ratio(location, string) for string in reference_list]
    max_index = max(range(len(fuzz_list)), key=fuzz_list.__getitem__)
    interpolated = reference_list[max_index] if fuzz_list[max_index] >= 40 else "user check"
    return interpolated

In [56]:
location = "Vp&danforth"
reference_list = ['yonge&steeles','yorkmillsstation','kiplingstation','victoriapark&danforth']
print(synchronize_locations(location, reference_list))

victoriapark&danforth


In [76]:
df_bus_delay['synchronized'] = df_bus_delay['copy'].apply(synchronize_locations, reference_list=stops['cleaned_stops'])

KeyboardInterrupt: 

In [59]:
df_bus_delay.head(20)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy,synchronized
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation,yorkmillsstation
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute,user check
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden,user check
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation,kiplingstation
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark,victoriapark&danforth
5,129,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown,user check
6,36,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot,user check
7,53,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,markham&passmore,user check
8,36,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation,kiplingstation
9,320,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge,user check
